In this notebook we will try to implement **Sign/Action Language Detection** using *Python*, *Tensorflow* and *MediaPipe Hoslistic Pipelines*.

Actually here we will be focussing on sequence of actions rather than a single frame to detect the sign. So the model or architecture of choice would be **Long Short Term Memory(LSTM)**.

[MediaPipe Holistic](https://github.com/google/mediapipe/blob/master/docs/solutions/holistic.md)

[LSTM](https://en.wikipedia.org/wiki/Long_short-term_memory)

# Install Required Dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe
# Also sklearn and matplotlib if not present already
# !pip install sklearn matplotlib

  Using cached tensorflow-2.4.1-cp38-cp38-win_amd64.whl (370.7 MB)


# Import Dependencies

In [88]:
import cv2 # open-cv for use of webcam etc
import numpy as np # store keyoints as arrays and mathematical manipulations
import os # work with file-system

import matplotlib.pyplot as plt # visualisation
import time

import mediapipe as mp # for extractiong keypoints from the live feed

from sklearn.model_selection import train_test_split # split data

# evaluating the model
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

from tensorflow.keras.utils import to_categorical # create categories

from tensorflow.keras.models import Sequential # sequential neural network
from tensorflow.keras.layers import LSTM, Dense # for LSTM and dense layers
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint # callback for monitoring and saving model

# Keypoints using MP Holistic
In this we use MediaPipe Holistics to capture Keypoints from our feed. Also, use MediaPipe drawings to embed keypoints over our feed.

In [11]:
# MediaPipe Holistic Model to capture keypoints
mp_holistic = mp.solutions.holistic

# Utilities to draw keypoints over our feed
mp_drawing = mp.solutions.drawing_utils

In [12]:
def mediapipe_detection(image, model):
    # convert input image from open-cv from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # image is non-writable(for memory management)
    image.flags.writeable = False 
    
    # detection over passed frame(image) using MediaPipe
    results = model.process(image)
    
    # image is writable
    image.flags.writeable = True
    
    # convert image from RGB to BGR for open-cv
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    return image, results

In [42]:
def draw_detected_landmarks(image, results):
    # draw face detections using ladnmarks and the connections for each landmark
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    
    # draw pose detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    
    # draw left-hand detections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    
    # draw right-hand detections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

# Test Webcam Feed
In this we will test the webcam feed with detected pose, face, hands using MediaPipe.

In [43]:
cap = cv2.VideoCapture(0) # access webcam

# access the MediaPipe Holistic model
with mp_holistic.Holistic() as holistic_model:
    while cap.isOpened(): # loop through all frames

        ret, frame = cap.read() # read the frame from webcam

        # Make detections using MediaPipe
        image, results = mediapipe_detection(frame, holistic_model)
        
        # draw/embed landmarks over the detected image poses
        draw_detected_landmarks(image,results)

        cv2.imshow('OpenCV Feed', image) # show on screen

        if cv2.waitKey(10) & 0xFF == ord('q'): # break from loop reading the frame
            break

    cap.release() # release webcam device
    cv2.destroyAllWindows() # close the frames

# Extract Key-Points
In this we will extract key-points from detected frames. Also, handle for the cases where we don't get any detections by blank arrays.

In [44]:
def extract_keypoints(results):
    # for pose-detection
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    # for face-detection
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    
    # for left-hand-detection
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    # for right-hand-detection
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Base Setup

In [111]:
# path for the exported data
DATA_PATH = os.path.join('MP_Data')

# actions to be detected
actions = np.array(['hello','thanks','iloveyou'])

# no of sequences to be considered for detcetion of an action(videos)
no_sequences = 30

# each video length(frames)
sequence_length = 30

In [112]:
# make folders for each action and each sequence
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            print('Exception Occured')
            pass

# Collect Keypoint Values(Data Collection)
In this we will collect data for each action.

In [113]:
cap = cv2.VideoCapture(0) # access webcam

# access the MediaPipe Holistic model
with mp_holistic.Holistic() as holistic_model:
    
    for action in actions: # each action
        for sequence in range(no_sequences): # each sequence
            for frame_num in range(sequence_length): # each frame

                ret, frame = cap.read() # read the frame from webcam

                # Make detections using MediaPipe
                image, results = mediapipe_detection(frame, holistic_model)

                # draw/embed landmarks over the detected image poses
                draw_detected_landmarks(image,results)
                
                # Apply Wait Logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    # show on screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000) # wait for 2 sec
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                    # show on screen
                    cv2.imshow('OpenCV Feed', image)
                    
                keypoints = extract_keypoints(results) # Extract Keypoints
                
                # save a numpy array in each action and sequence folder
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'): # break from loop reading the frame
                    break

    cap.release() # release webcam device
    cv2.destroyAllWindows() # close the frames

# Data Pre-Processing
In this we will do a `train-test split` of the data. Also, create labels and features for the deep neural network.

In [114]:
# create label map
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [115]:
sequences, labels = [], [] # feature data(X), labels(Y)

for action in actions: # each action
    for sequence in range(no_sequences): # each sequence
        window = [] # all frames for a sequence
        for frame_num in range(sequence_length): # each frame
            # load numpy based frames
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [176]:
# features
X = np.array(sequences)
X.shape

(90, 30, 1662)

In [177]:
# labels
y = to_categorical(labels).astype(int)
y.shape

(90, 3)

In [221]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Build and Train LSTM Neural Network

In [222]:
log_dir = os.path.join('Logs') # setup log directory
tb_callback = TensorBoard(log_dir=log_dir) # monitor using TensorBoard callback

# NOTE: use tensorboard extension or command line
# %load_ext tensorboard
# %tensorboard --logdir <<log-directory>>

# tensorboard --logdir=<<log-directory>>

In [223]:
# Saving the best model using model checkpoint callback
model_checkpoint = ModelCheckpoint('action_net.h5', 
                                    save_best_only=True, 
                                    monitor='categorical_accuracy', 
                                    mode='max', 
                                    verbose=1
                                  )

In [229]:
# Building the neural network
model = Sequential() # initialise the sequential model

# have multi-layer LSTM(stacked)
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

# use Dense layers(fully-connected layers)
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# multi-class classification layer for actions
model.add(Dense(actions.shape[0], activation='softmax'))

In [230]:
# compile the model with Adam optimizer, categorical cross entropy
model.compile(optimizer='Adam', 
              loss='categorical_crossentropy', 
              metrics=['categorical_accuracy']
             )

In [231]:
# check model architecture
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_39 (LSTM)               (None, 30, 64)            442112    
_________________________________________________________________
lstm_40 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
lstm_41 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_39 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_40 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_41 (Dense)             (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
_______________________________________________

In [243]:
#  fit the model
model.fit(X_train, y_train, 
          epochs=2000, 
          callbacks=[tb_callback, model_checkpoint]
         )

# Load Saved Model, Make Predictions, Evaluate

In [232]:
# load saved weights to the model
model.load_weights('action_net.h5')

In [233]:
# predict on test data
y_pred = model.predict(X_test)
y_pred

array([[2.9792609e-03, 3.2631186e-05, 9.9698812e-01],
       [9.9938071e-01, 2.1581777e-12, 6.1929988e-04],
       [9.9194777e-11, 9.9999976e-01, 2.8206767e-07],
       [9.9930382e-01, 1.8642418e-14, 6.9621176e-04],
       [9.9897718e-01, 1.0028432e-11, 1.0227843e-03],
       [1.0539658e-09, 9.9997330e-01, 2.6754913e-05],
       [4.7263089e-11, 1.0000000e+00, 1.4124115e-08],
       [4.7726557e-02, 3.8103924e-05, 9.5223540e-01],
       [9.2308837e-01, 4.0486885e-09, 7.6911665e-02]], dtype=float32)

In [234]:
actions[np.argmax(y_pred[0])] # predicted

'iloveyou'

In [235]:
actions[np.argmax(y_test[0])] # actual

'iloveyou'

In [236]:
y_true = np.argmax(y_test, axis=1).tolist()
y_hat = np.argmax(y_pred, axis=1).tolist()

In [237]:
y_true

[2, 0, 1, 0, 0, 1, 1, 2, 0]

In [238]:
y_hat

[2, 0, 1, 0, 0, 1, 1, 2, 0]

In [239]:
# get the confusion matrix
multilabel_confusion_matrix(y_true, y_hat)

array([[[5, 0],
        [0, 4]],

       [[6, 0],
        [0, 3]],

       [[7, 0],
        [0, 2]]], dtype=int64)

In [240]:
# accuracy
accuracy_score(y_true,y_hat)

1.0

# Test on Live Feed

In [241]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [244]:
sequence = [] # sequence of 30 frames
sentence = [] # text
predictions = []
threshold = 0.4 # confidence threshold

cap = cv2.VideoCapture(0) # access webcam

# access the MediaPipe Holistic model
with mp_holistic.Holistic() as holistic_model:
    while cap.isOpened(): # loop through all frames

        ret, frame = cap.read() # read the frame from webcam

        # Make detections using MediaPipe
        image, results = mediapipe_detection(frame, holistic_model)

        # draw/embed landmarks over the detected image poses
        draw_detected_landmarks(image,results)

        # Prediction Logic
        keypoints = extract_keypoints(results) # Extract Keypoints
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        # predict after 30 frames/sequences
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            # print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
          
            # Visualisation logic 
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 

                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Visualisation probabilities
            image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    1, (255, 255, 255), 2, cv2.LINE_AA)

        # show on screen
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'): # break from loop reading the frame
            break

    cap.release() # release webcam device
    cv2.destroyAllWindows() # close the frames